# 目标优化

此笔记本演示了 Amazon Personalize 目标优化的一个用例。通常还有其他因素影响推荐，目标优化可用于提供额外数据来驱动模型。

此示例假设我们正在为视频点播（VOD）流媒体订阅服务提供推荐。此服务允许用户无限制地访问视频目录，但根据与内容所有者的安排，目录中可用的标题具有不同的许可条款。通过提供推荐，以较低的许可成本为内容提供权重，流媒体服务可以降低许可成本，同时仍能让客户满意。

同样，数据来自 [MovieLens](https://movielens.org/) 项目，您可以通过在下面单元格中的任何等待期间进行 Web 搜索来了解有关数据和潜在用途的更多信息。您将使用额外的特许权字段补充 MovieLens 数据，并生成将向数据集提供特许权使用费的数据。 

## 如何使用笔记本

代码被分解为如下所示的单元格。在本页顶部有一个三角形 `Run` 按钮，您可以单击该按钮以执行每个单元格，并移至下一个单元格；也可以按 `Shift` + `Enter` 组合键，在单元格中执行并移至下一个单元格。

当一个单元格执行时，您会注意到该单元格运行时旁边有一行显示 `*`，或者它会更新为一个数字，以指示在执行完单元格中的所有代码后完成执行的最后一个单元格。


只需按照以下说明进行操作，并执行单元格即可开始 Amazon Personalize 目标优化。

## 导入 

Python 附带有许多库，我们需要导入这些库以及已安装的库来帮助我们，比如 [boto3](https://aws.amazon.com/sdk-for-python/)（适用于 Python 的 AWS SDK）和 [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/)，它们都是核心数据科学工具。

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
from botocore.exceptions import ClientError

!conda install -y -c conda-forge unzip

pd.options.mode.chained_assignment = None

接下来，您需要验证您的环境是否可以成功地与 Amazon Personalize 进行通信，下面的行就可以实现这一目的。

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## 配置数据

数据通过 Amazon S3 导入 Amazon Personalize，下面我们将指定一个文件名，用于在将文件上载到 S3 之前在本地写入这些文件。


In [ ]:
filename = "movie-lens-100k.csv"
items_filename = "movie-lens-items.csv"

### 下载、准备和上载训练数据

目前您尚未在本地加载 MovieLens 数据进行检查，请执行以下行，下载最新副本并快速检查。

#### 下载并浏览数据集

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

#### 准备和上载数据

以下代码可从 MovieLens 下载中加载电影信息。它提供有关标题、发布日期、IMDB 链接的信息，以及用于捕获适用于标题的流派的列列表。

由于研讨会正在生成虚构的特许权，因此没有特许权字段，将在下一步中完成此操作。








In [ ]:
items = pd.read_csv('./ml-100k/u.item', sep='|',encoding='latin-1', names=['ITEM_ID', 'TITLE', 'RELEASE_DATE', 'VIDEO_RELEASE_DATE', 'IMDB_URL', 'MISC', 'ACTION_GENRE', 'ADVENTURE_GENRE', 'ANIMATION_GENRE', 'CHILDRENS_GENRE','COMEDY_GENRE', 'CRIME_GENRE', 'DOCUMENTARY_GENRE','DRAMA_GENRE','FANTASY_GENRE', 'FILMNOIR_GENRE','HORROR_GENRE', 'MUSICAL_GENRE','MYSTERY_GENRE', 'ROMANCE_GENRE', 'SCIFI_GENRE', 'THRILLER_GENRE', 'WAR_GENRE', 'WESTERN_GENRE'     ])
items

#### 添加特许权数据和流派
我们正在为 ROYALTY 字段分配一个值，该值分布均匀，分别为 0.0、0.005、0.01、0.015、0.02、0.025、0.05、0.10。这使得大多数电影的特许权使用费相对较低或为零，而少数电影的特许权使用费较高。

请注意下面的条形图，标题与每个特许权使用费均匀分布。

上面的 MovieLens GENRE 字段结果已分类，每一列都指示标题是否属于该流派，但加载到个性化设置中的项目可以接受数组。一些 Pandas 代码，可将这些流派合并到一个列出所有流派的以管道分隔的列中。



In [ ]:
pd.set_option('display.max_rows', 10)

royaltyvalues = [0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.05, 0.10]
items.loc[:,'ROYALTY'] = items['ITEM_ID'].map(lambda x: royaltyvalues[x%8])

items.loc[:,'GENRE']='' 

for col_name in items.columns:
    if col_name.endswith('_GENRE'):
        items.loc[items[col_name]==1,'GENRE']= items['GENRE']+'|'+ col_name[:-6]

items = items[['ITEM_ID', 'TITLE','ROYALTY', 'GENRE']]
items.loc[:,'GENRE'] = items['GENRE'].str[1:]


items.loc[:,'ROYALTY'].value_counts().plot.bar()
items.head(10)


#### 将特许权值调整为负数。

目标优化将优化到字段的最高值。在这种情况下，我们希望平衡电影的相关性与流媒体服务将产生的特许权费用。

为了提升特许权使用费最低的电影，特许权使用费最低的电影需要最高的数值。在这种情况下，我们会将特许权转换为负数。此值是通过将 -1 乘以特许权使用费的绝对值得出的。

注意：使用绝对值，因此该单元格可以运行两次并获得相同的结果，这在研讨会环境中很有用。

In [ ]:
items.loc[:,'ROYALTY'] = -1 * abs(items['ROYALTY'])
items

## 配置 S3 存储桶和 IAM 角色<a class="anchor" id="bucket_role"></a>
到目前为止，我们已下载、操作数据并将数据保存到运行此 Jupyter 笔记本的实例所连接的 Amazon EBS 实例中。但是，Amazon Personalize 需要一个 S3 存储桶充当您的数据源以及用于访问该存储桶的 IAM 角色。让我们把一切都安排好。

使用存储在此 Amazon SageMaker 笔记本底层的实例上的元数据来确定运行区域。如果您使用的是 Amazon SageMaker 以外的 Jupyter 笔记本，只需将区域定义为下面的字符串。Amazon S3 存储桶需要与我们目前创建的 Amazon Personalize 资源位于同一区域。

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    notebook_data = json.load(notebook_info)
    resource_arn = notebook_data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

Amazon S3 存储桶名称是全局唯一的。要创建唯一的存储桶名称，下面的代码会将字符串 `personalize-objective-optimization-` 附加到您的 AWS 账号。然后，它会使用此名称在前一个单元格中发现的区域创建一个存储桶。

In [ ]:
s3 = boto3.client('s3')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-objective-optimization-"+   suffix        # replace with the name of your S3 bucket
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

### 将数据上载到 S3

现在，您的 Amazon S3 存储桶已创建，请上载我们的项目数据的 CSV 文件。 

In [ ]:
items = items[['ITEM_ID', 'TITLE', 'GENRE', 'ROYALTY']]
items_dataset = items[['ITEM_ID','ROYALTY', 'GENRE']]

items_dataset.to_csv(items_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_filename).upload_file(items_filename)

### 设置 S3 存储桶策略
Amazon Personalize 需要能够读取 S3 存储桶的内容。因此，添加可实现此操作的存储桶策略。

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

#### 创建电影标题项目架构
我们将需要两个架构，一个用于电影标题，它是类型是 Items（项目），另一个是用于定义交互的结构。

In [ ]:
schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "ROYALTY",
            "type": "float"
        },        {
            "name": "GENRE",
            "type": [
                "null",
                "string"
              ],
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_item_schema_response = personalize.create_schema(
    name = "personalize-objective-optmization-item-schema"+suffix,
    schema = json.dumps(schema)
)

item_schema_arn = create_item_schema_response['schemaArn']
print(json.dumps(create_item_schema_response, indent=2))

从这一点开始，我们将遵循与[第一个活动](https://github.com/aws-samples/amazon-personalize-samples/blob/master/getting_started/notebooks/1.Building_Your_First_Campaign.ipynb)相同的流程，从而加载交互。我们只希望包含评级至少为 3 的交互，我们不希望推荐观众不喜欢的电影。

In [ ]:
data = data[data['RATING'] > 3]                # Keep only movies rated higher than 3 out of 5.
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(filename).upload_file(filename)

### 创建交互式架构

Personalize 如何理解您的数据的核心组件来自下面定义的模式。此配置指导服务如何消化通过 CSV 文件提供的数据。注意列和类型与您在上面创建的文件中的内容对齐。

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-objective-optmization-schema"+suffix,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

### 创建并等待数据集组

Personalize 中最大的分组是数据集组，这将隔离您的数据、事件跟踪器、解决方案和活动。把共享通用数据集合的内容分组在一起。您可以随意更改下面的名称。

#### 创建数据集组

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-objective-optmization-demo-"+suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### 等待数据集组状态变为 ACTIVE

在我们可以在以下任何项目中使用数据集组之前，数据集组必须处于活跃状态，请执行下面的单元格并等待项目显示为活跃状态。

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### 创建数据集

在创建组之后，接下来要创建的是实际数据集，在本例中，我们将为交互数据创建 1 个数据集，为项目数据创建另一个数据集。执行以下单元格以创建数据集。

In [ ]:
def create_dataset(dataset_type, schema_arn, name):
    create_dataset_response = personalize.create_dataset(
        name = name,
        datasetType = dataset_type,
        datasetGroupArn = dataset_group_arn,
        schemaArn = schema_arn
    )
    dataset_arn = create_dataset_response['datasetArn']

    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_dataset_response = personalize.describe_dataset(
            datasetArn = dataset_arn
        )
        status = describe_dataset_response["dataset"]["status"]
        print("Dataset: {} {}".format(name, status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(10)
    return dataset_arn


In [ ]:
interaction_dataset_arn = create_dataset("INTERACTIONS", schema_arn, 'personalize-objective-optmization-interactions-'+suffix)


print('interaction_dataset_arn: ' + interaction_dataset_arn)


In [ ]:
item_dataset_arn = create_dataset("ITEMS", item_schema_arn, 'personalize-objective-optmization-items-'+suffix)
print('item_dataset_arn: ' + item_dataset_arn)

#### 创建 Personalize 角色

此外，Amazon Personalize 需要能够担任 AWS 中的角色，以便拥有执行某些任务的权限，下面的行授予了此权限。

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## 导入数据

您之前创建了数据集组和数据集来存储您的信息，现在您将执行一个导入作业，从 S3 将数据加载到 Amazon Personalize 中，用于构建您的模型。我们将加载多个导入项，因此将调用下面的函数来启动导入作业，然后监控导入作业是否完成。

#### 创建数据集导入作业

In [ ]:
def create_dataset_import_job(dataset_arn, dataLocation, name):
    create_dataset_import_job_response = personalize.create_dataset_import_job(
        jobName = name,
        datasetArn = dataset_arn,
        dataSource = {
            "dataLocation": dataLocation
        },
        roleArn = role_arn
    )

    dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
    
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = dataset_import_job_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        print("DatasetImportJob: {} {}".format(name, status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(60)
    return dataset_import_job_arn

#### 加载交互
下面的导入作业将加载交互数据集。

In [ ]:
dataset_import_job_arn = create_dataset_import_job(interaction_dataset_arn, "s3://{}/{}".format(bucket_name, filename), "personalize-objective-optimization-interaction-"+suffix)
print('dataset_import_job_arn: ' + dataset_import_job_arn)

#### 将电影标题加载到项目数据集中

导入作业可能需要一会儿才能完成，请等待直至看到它在下面处于活跃状态。

In [ ]:
item_dataset_import_job_arn = create_dataset_import_job(item_dataset_arn, "s3://{}/{}".format(bucket_name, items_filename), "personalize-objective-optimization-item-"+suffix)
print('item_dataset_import_job_arn: ' + item_dataset_import_job_arn)

## 创建解决方案和版本

在 Amazon Personalize 中，经过训练的模型称为“解决方案”。模型经过训练后，每个解决方案都可以有许多与给定数据量相关的特定版本。

首先，我们将列出受支持的所有配方，配方是一种尚未接受过数据训练的算法。列出后，您将选择一个配方并使用它来构建您的模型。

### 选择配方

#### 用户个性化
[User-Personalization](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html)（aws-user-personalization）配方针对所有 USER_PERSONALIZATION 推荐方案进行了优化。在推荐项目时，它使用自动项目探索。

借助自动探索功能，Amazon Personalize 自动测试不同的项目推荐，了解用户如何与这些推荐项目交互，并改进可提高参与度和转化率的项目的推荐。当您的目录快速变化，或者当新项目（如新闻文章或促销）与新用户更相关时，这可改进项目发现和参与度。

您可以平衡探索的量（推荐更频繁地探索较低交互数据或相关性较低的项目）与要利用的量（推荐基于我们的了解内容或相关性）。Amazon Personalize 根据隐式用户反馈自动调整未来的推荐。

首先，通过在上面的配方列表中找到 ARN 来选择配方。

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization" # aws-user-personalization selected for demo purposes

### 创建并等待解决方案

首先，您将使用 API 创建解决方案，然后创建一个版本。系统将需要几分钟时间来训练模型并创建解决方案的版本。开始后，您会收到通知告知您操作已在进行中，这样您就可以休息一下了，比如喝杯咖啡等。

此函数接受客观灵敏度设置，可以是 OFF（关闭）、LOW（低）、MED（中等）或 HIGH（高）。这将调整将目标影响驱动到模型中的权重。

此函数可创建解决方案以及该解决方案的初始解决方案版本。

#### 创建解决方案

In [ ]:
def create_solution(name, objectiveSensitivity):
    create_solution_response = personalize.create_solution(
        name = name,
        datasetGroupArn = dataset_group_arn,
        recipeArn = recipe_arn,
        solutionConfig = {
            "optimizationObjective": {
                "itemAttribute": "ROYALTY",
                "objectiveSensitivity":objectiveSensitivity
            }
        }
    )
    
    solution_arn = create_solution_response['solutionArn']
    
    print('solutionArn:' + solution_arn)

    create_solution_version_response = personalize.create_solution_version(
        solutionArn = solution_arn
    )

    solution_version_arn = create_solution_version_response['solutionVersionArn']
    print('solution_version_arn: ' + solution_version_arn)

    return {
        "solution_arn": solution_arn,
        "solution_version_arn": solution_version_arn
    }
    
def waitForSolutionVersion(solution_version_arn):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_solution_version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = describe_solution_version_response["solutionVersion"]["status"]
        print("SolutionVersion: {} {}".format(solution_version_arn, status))

        if status == "ACTIVE" or status == "CREATE FAILED"  or status == "CREATE STOPPING":
            break

        time.sleep(60)

#### 创建解决方案版本
创建 3 个不同的解决方案，一个关闭目标优化，一个将目标优化设置为 LOW（低），一个将目标优化设置为 HIGH（高）。这将并行启动 3 个解决方案，下一步就是等待它们完成。

In [ ]:
high_solution = create_solution('movie-recommendation-low-royalties-'+suffix, 'HIGH')
low_solution = create_solution('movie-recommendation-medium-royalties-'+suffix, 'LOW')
no_objective_optimization_solution = create_solution('movie-recommendation-max-relevance-'+suffix, 'OFF')

#### 等待解决方案版本处于 ACTIVE 状态

这大约需要 40-50 分钟。

In [ ]:
waitForSolutionVersion(low_solution['solution_version_arn'])
waitForSolutionVersion(no_objective_optimization_solution['solution_version_arn'])
waitForSolutionVersion(high_solution['solution_version_arn'])

#### 获取解决方案版本的指标

既然您的解决方案和版本已经存在，您就可以获取指标来判断性能。这些指标不是特别好，因为它是一组演示数据，但如果是更大更复杂的数据集，您应该会看到改进。

您可以看到模型质量受目标优化影响而产生的差异

In [ ]:
def get_solution_metrics(solutions):
    metricdata = { "name": []}
    
    for key in solutions:
        solution = solutions[key]
        
        metricdata["name"].append(key)
        
        get_solution_metrics_response = personalize.get_solution_metrics(
            solutionVersionArn = solution['solution_version_arn']
        )

        for metricname in get_solution_metrics_response['metrics']:
            if not metricname in metricdata:
                metricdata[metricname] = []
                
            metricdata[metricname].append( get_solution_metrics_response['metrics'][metricname])
            
        # print(json.dumps(get_solution_metrics_response, indent=2))
    return pd.DataFrame.from_dict(metricdata);

metrics = get_solution_metrics({
    "no-optimization": no_objective_optimization_solution,
    "low-optimization": low_solution,
    "high-optimization": high_solution,
})

metrics

我们建议阅读[文档](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)了解指标，但为了方便起见，我们还复制了以下文档的部分内容。

您需要了解 Personalize 中有关评估的以下术语：

- *相关推荐*是指与特定用户测试数据中的值相匹配的推荐。
- *排名*是指推荐清单中推荐项目的位置。位置 1（列表顶部）假定与用户最相关。
- *查询*是指 GetRecommendations 调用的内部等效项。

Personalize 生成的指标包括：

- 覆盖范围：训练数据（包括项目和交互数据集）中来自所有查询的唯一推荐项目数占唯一项目总数的比例。
- mean_reciprocal_rank_at_25：对所有查询提出的前 25 项推荐中，第一项相关推荐的[倒数排名的平均值](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)。如果您对单个排名最高的推荐感兴趣，则此指标是合适的。
- normalized_discounted_cumulative_gain_at_K：折损增益假设推荐列表中较下面的推荐的相关性低于较上面的推荐。因此，每项推荐都被一个因数折损（为推荐赋予较低的权重），具体取决于推荐的位置。为了产生 K 处的[累积折损增益](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)（DCG），将前 K 个推荐中的每个相关折损推荐加在一起。归一化折损累积增益（NDCG）是 DCG 除以理想 DCG，使得 NDCG 在 0-1 之间。（理想 DCG 是按相关性对前 K 个推荐进行排序。） Amazon Personalize 使用 1/log(1 + position) 权重因子，其中列表顶部为位置 1。此指标奖励出现在列表顶部附近的相关项目，因为列表的顶部通常会更受关注。
- precision_at_K：前 K 个推荐中相关推荐的数量除以 K。此指标奖励相关项目的精确推荐。

## 创建并等待活动

现在您有了一个有效的解决方案版本，您需要创建一个活动，以便用于您的应用程序。活动是托管解决方案版本；可以查询推荐的端点。通过估计吞吐量（用户每秒的个性化请求）来设置定价。部署活动时，您可以设置每秒事务数（TPS）最小值（`minProvisionedTPS`）。与 AWS 中的许多服务一样，此服务将根据需求自动扩展，但如果延迟至关重要，您可能希望提前预置以满足更大的需求。对于此演示，最小吞吐量阈值设置为 1。有关更多信息，请参阅[定价](https://aws.amazon.com/personalize/pricing/)页面。

如上所述，我们的解决方案使用的 user-personalization 配方支持自动浏览"冷"项目。您可以控制在创建活动时执行的浏览量。`itemExplorationConfig` 数据类型支持 `explorationWeight` 和 `explorationItemAgeCutOff` 参数。浏览权重决定推荐包含交互较少数据或相关性较低项目的频率。值越接近 1.0，则浏览量越多。在为零时，不会进行浏览，推荐基于当前数据（相关性）。浏览项目期限截止日期根据自上次交互以来的时间范围确定要浏览的项目。提供自上次交互以来的最大项目期限（以天为单位），以定义项目浏览的范围。值越大，在浏览过程中考虑的项目越多。对于以下活动，我们将指定 0.5 的浏览权重。

#### 创建活动

In [ ]:
def create_campaign(solution, name):
    create_campaign_response = personalize.create_campaign(
        name = "personalize-demo-" + name + '-' + suffix,
        solutionVersionArn = solution['solution_version_arn'],
        minProvisionedTPS = 1,
        campaignConfig = {
            "itemExplorationConfig": {
                "explorationWeight": "0.5"
            }
        }
    )

    campaign_arn = create_campaign_response['campaignArn']
    print('campaign_arn:' + campaign_arn)
    return campaign_arn

def waitForCampaign(solution):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_campaign_response = personalize.describe_campaign(
            campaignArn = solution['campaign_arn']
        )
        status = describe_campaign_response["campaign"]["status"]
        print("Campaign: {} {}".format(solution['campaign_arn'], status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(60)

#### 创建 3 个活动
为每个目标优化创建一个活动，但保持所有其他设置不变，以演示目标优化的影响。

In [ ]:
high_solution['campaign_arn'] = create_campaign(high_solution, 'high')
low_solution['campaign_arn'] = create_campaign(low_solution, 'low')
no_objective_optimization_solution['campaign_arn'] = create_campaign(no_objective_optimization_solution, 'max_relevance')


#### 等待活动状态变为 ACTIVE

这大约需要 10 分钟。

In [ ]:
waitForCampaign(high_solution)
waitForCampaign(low_solution)
waitForCampaign(no_objective_optimization_solution)

## 获取样本推荐

活动状态变为活跃后，您就可以获得推荐。首先，我们需要从集合中选择一个随机用户。然后，我们将创建一些辅助函数，以获取电影信息以显示推荐，而不是仅显示 ID。

In [ ]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.iloc[movie_id]['TITLE'] + '(' + f'{-1*items.iloc[movie_id]["ROYALTY"]:.2f}'+ ')'

def get_movie_royalty(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return -1*items.iloc[movie_id]["ROYALTY"]

#### 调用 GetRecommendations

使用您在上面获得的用户，以下行将为您提供推荐，并返回推荐的电影列表。


In [ ]:
def get_recommendations(solution):
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = solution['campaign_arn'],
        userId = str(user_id),
    )
    # Update DF rendering
    pd.set_option('display.max_rows', 30)

    item_list = get_recommendations_response['itemList']

    recommendation_list = []

    total_royalties = 0.0
    
    for item in item_list:
        title = get_movie_title(item['itemId'])
        total_royalties = total_royalties + get_movie_royalty(item['itemId'])
        recommendation_list.append(title)
        
    recommendation_list.append('TOTAL ROYALTIES: '+ f'{total_royalties:.2f}')
    return recommendation_list

#### 比较推荐
为同一用户创建一组推荐，以比较目标优化的影响。

请注意标题和年份后的括号中显示的特许使用费值的影响。关闭目标优化的高评分、高特许权电影往往会在目标优化开启时显示在列表中的较低位置（如果显示）。

还要注意每组推荐中所有标题的总特许权。

In [ ]:
recommendations_df = pd.DataFrame(get_recommendations(no_objective_optimization_solution), columns = ['ObjectiveOff'])
recommendations_df['LowObjective'] = get_recommendations(low_solution)
recommendations_df['HighObjective'] = get_recommendations(high_solution)
recommendations_df

## 审核

此笔记本展示了一个示例，其中的电影推荐引擎将需要为给定标题支付特许权使用费纳入考察范围。通过在推荐算法中加入此权重，流媒体服务可以为用户提供良好的推荐，但也可以将支付给内容创建者的特许权使用费降到最低。

请注意，在上图中，在关闭目标优化的情况下，括号中的特许权使用费是相当均匀地分配的，正如人们所期望的那样，因为它没有被考虑在内。但是，在 LowObjective 列中，最高目标优化设置的值较低，特许权使用费总和最低。

## 清理

清理资源

In [ ]:
def delete_campaign(campaign_arn):
    delete_campaign_result = personalize.delete_campaign(campaignArn=campaign_arn )
    

def wait_for_delete_campaign(campaign_arn):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        try:
            describe_campaign_response = personalize.describe_campaign(
                campaignArn = campaign_arn
            )
            status = describe_campaign_response["campaign"]["status"]
            print("campaign: {}".format(status))

        except ClientError as e:
            print(e)
            break

        time.sleep(10)
    print('campaign ' + campaign_arn + ' deleted')
    
def delete_solution(solution_arn):
    delete_solution_result = personalize.delete_solution(solutionArn=solution_arn )
    
    max_time = time.time() + 3*60*60 # 3 hours

def wait_for_delete_solution(solution_arn):
    while time.time() < max_time:
        
        try:
            describe_solution_response = personalize.describe_solution(
                solutionArn = solution_arn
            )
            status = describe_solution_response["solution"]["status"]
            print("Solution: {}".format(status))

        except ClientError:
            break
        time.sleep(10)
    print('Solution ' + solution_arn + ' deleted')
    
def delete_dataset(dataset_arn):
    delete_dataset_result = personalize.delete_dataset(datasetArn=dataset_arn )
    
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        try:
            describe_dataset_response = personalize.describe_dataset(
                datasetArn = dataset_arn
            )
            status = describe_dataset_response["dataset"]["status"]
            print("dataset: {}".format(status))

        except ClientError:
            break
        time.sleep(10)
    print('dataset ' + dataset_arn + ' deleted')
    
def delete_schema(schema_arn):
    delete_schema_result = personalize.delete_schema(schemaArn=schema_arn )
    

    print('schema ' + schema_arn + ' deleted')
    
def delete_dataset_group(dataset_group_arn):
    delete_dataset_group_result = personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn )
    
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        try:
            describe_dataset_group_response = personalize.describe_dataset_group(
                datasetGroupArn = dataset_group_arn
            )
            status = describe_dataset_group_response["datasetGroup"]["status"]
            print("dataset_group: {}".format(status))

        except ClientError:
            break
        time.sleep(10)
    print('dataset_group ' + dataset_group_arn + ' deleted')
    
def delete_all(solutions):
    for solution in solutions:
        delete_campaign(solution['campaign_arn'])
        
    for solution in solutions:
        wait_for_delete_campaign(solution['campaign_arn'])
    for solution in solutions:
        delete_solution(solution['solution_arn'])
    for solution in solutions:
        wait_for_delete_solution(solution['solution_arn'])


    

In [ ]:
delete_all([no_objective_optimization_solution, low_solution, high_solution] )

In [ ]:
delete_dataset(item_dataset_arn)
delete_dataset(interaction_dataset_arn)

In [ ]:
delete_schema(item_schema_arn)
delete_schema(schema_arn)

In [ ]:
delete_dataset_group(dataset_group_arn)

In [ ]:
iam = boto3.client("iam")

iam.detach_role_policy(RoleName=role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess')
iam.detach_role_policy(RoleName=role_name, PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess')
time.sleep(10) # propogation time

iam.delete_role(RoleName=role_name)

In [ ]:
! aws s3 rm --recursive s3://$bucket_name

In [ ]:
s3.delete_bucket(Bucket=bucket_name)